# Create Training Files - Step 1

In [79]:
import pandas as pd
import json

In [96]:
def create_jsonl(load_file, save_file):
    # load CSV file
    df = pd.read_csv(load_file)
    print(f"File {load_file.split('/')[(len(load_file.split('/'))) - 1]} contains {len(df)} rows.")
    # put training data in the right prompt format - chat-style training is used
    training_data = []
    for i in range(len(df)):
        dialogue = df['dialogue'][i]
        note = df['note'][i]
        training_data.append({"messages": [{"role": "system", "content": "You are an expert medical professional. Given a clinical dialogue, create a clinical note outlining key dialogue aspects such as 'CHIEF COMPLAINT' (or 'CC'), 'HISTORY OF PRESENT ILLNESS' ('HPI'), 'REVIEW OF SYSTEMS', 'PHYSICAL EXAMINATION', 'VITALS REVIEWED', 'RESULTS', 'ASSESSMENT AND PLAN', 'INSTRUCTIONS', 'CURRENT MEDICATIONS', 'PAST MEDICAL HISTORY', 'EXAM', 'IMPRESSION', 'PLAN', 'ASSESSMENT', 'PAST HISTORY', 'ALLERGIES', 'SOCIAL HISTORY', 'PHYSICAL EXAM', 'PROCEDURE', 'FAMILY HISTORY', 'MEDICATIONS', 'VITALS', 'MEDICAL HISTORY', 'SURGICAL HISTORY'. You will not use all of these aspects in every dialogue, vary it from dialogue to dialogue."}, {"role": "user", "content": f"Dialogue:{dialogue}"}, {"role": "assistant", "content": f"{note}"}]})
    # save in the required file format
    if not '.jsonl' in save_file:
        save_file = save_file + '.jsonl'
    with open(f"./tuning_files/{save_file}", "w") as f:
        for item in training_data:
            f.write(json.dumps(item) + "\n")


In [97]:
# create training data
create_jsonl("./clinical_visit_note_summarization_corpus/data/aci-bench/challenge_data/train.csv", "train.jsonl")
# create validation data
create_jsonl("./clinical_visit_note_summarization_corpus/data/aci-bench/challenge_data/valid.csv", "valid.jsonl")

File train.csv contains 67 rows.
File valid.csv contains 20 rows.
